# Julia言語で入門するプログラミング（おまけ）

## 構造体の再定義

Julia には無名構造体の構文が存在しないため、構造体を定義した場合、その構造体名は固定シンボルとなり、再定義ができない

もし無名構造体を定義可能なのであれば、以下のような形で、変数に無名構造体を束縛することで構造体の再定義のようなことが実現可能だった

```julia
# もし無名構造体を宣言できるなら・・・
キャラクター = struct
    HP::Int
end

# 変数に無名構造体を束縛すれば良いだけなので、構造体の再定義のようなことが実現可能
キャラクター = struct
    名前::String
    HP::Int
end
```

しかし、実際には無名構造体の宣言ができないため、上記のような書き方はできない

前回は、構造体の再定義を行うために、module を使ったトリックを使った

module は再定義可能（同名の module を定義した場合、古い module は上書きされる）のため、module ごと置換して構造体の再定義を行ってしまおうというアイディアである

```julia
# RPG モジュールの中で キャラクター 構造体を定義
module RPG
    struct キャラクター
        HP::Int
    end
end

# RPG モジュールを再定義
## => 前に定義していた RPG モジュールは消える
## => RPG モジュール内の キャラクター 構造体を再定義可能
module RPG
    struct キャラクター
        名前::String
        HP::Int
    end
end
```

しかし、これは module の本来の使い方ではないし、module 内に使いまわししたい関数や変数が存在した場合、取り回ししづらい

（module の再定義を行った場合、module 内の定義はすべて消えてしまうため、使いまわししたい関数や変数は一度別の変数に退避して、再定義した module 内に import しなおさなければならない）

そこで今回は、別のアイディアとして「適当な名前の構造体を一時定義して、使いたい名前の変数に束縛する」という形で、構造体の再定義を実現してみる

In [2]:
# 適当な名前で構造体を定義
struct _キャラクター構造体1
    HP::Int
end

# 定義した _キャラクター構造体1 を キャラクター 変数に束縛
キャラクター = _キャラクター構造体1

# キャラクター 変数は構造体として利用可能
勇者 = キャラクター(10)
dump(勇者)

_キャラクター構造体1
  HP: Int64 10


In [3]:
# 別の適当な名前で構造体を定義
struct _キャラクター構造体2
    名前::String
    HP::Int
end

# 定義した _キャラクター構造体2 を キャラクター 変数に束縛
キャラクター = _キャラクター構造体2

# => これで キャラクター 構造体を再定義したような形になる

勇者 = キャラクター("勇者", 10)
dump(勇者)

_キャラクター構造体2
  名前: String "勇者"
  HP: Int64 10


## メタプログラミング

上記のような書き方で構造体の再定義を行う場合、「適当な名前で構造体を定義する」という操作は、できれば Julia 自身にやってもらい自分で記述しなくても良いようにしたい

こういった場合のために、Julia には **メタプログラミング** のための構文が用意されている

すなわち「Julia が実行可能なコードを Julia 自身に生成させる」ための仕組みである

### Julia がコードを実行する仕組み
Julia に関わらず、すべてのプログラミング言語において、コードは文字列として表現される

この文字列が解析されて抽象構文木（AST）に分解され、さらにそれがコンピュータの命令文として解釈され、コードが実行される

この「文字列を AST に分解する」ための仕組みとして、Julia には `Meta.parse` 関数が提供されている

In [4]:
# コード: `a = 1 + 10`
code = "a = 1 + 10"

# コード => AST に分解
## Julia では AST は `Expr` オブジェクトとして定義されている
ast = Meta.parse(code)
dump(ast)

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol a
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 10


上記のように、コードの文字列はまず AST (`Expr` オブジェクト) に分解される

さらにこれを「コンピュータの命令文として解釈する」（＝コードを実行する）ための仕組みとして、Julia には `eval` 関数が提供されている

In [5]:
# AST を解釈して実行する
## => a = 1 + 10 が実行される
eval(ast)

11

In [6]:
# a = 1 + 10 が実行されたため、変数 a が生成されて、その値として 1 + 10 = 11 が束縛されている
a

11

### マクロ
上記のような流れでコードの分解・実行が行われるわけだが、この一連の流れをまとめて定義するための仕組みとして **マクロ** 機能がある

マクロは、AST を作成する関数のようなものであり、以下のような形で定義する

```julia
macro マクロ名(マクロに渡す引数...)
    # AST 作成 = Expr オブジェクトの構築
    # 構築した Expr オブジェクトを返す
end
```

これを使うと、「適当な名前で構造体を定義する」という操作（AST の作成）を実現することができる

「適当な名前」を生成するための関数として `gensym` 関数があるため、これを使うと以下のように書ける

In [12]:
macro structure()
    # 適当な名前を生成
    一時的な構造体の名前 = gensym()

    # 構造体定義の Expr オブジェクトを返す
    :(
        struct $一時的な構造体の名前
            HP::Int
        end
    )
end

# マクロ呼び出しは `@マクロ名` で行う
## => マクロが構築した Expr オブジェクトに対して eval 関数が呼び出されて実行される
@structure

#=> 分かりにくいが、以下のような構造体宣言が実行されている

# ※ Julia においてすべての「名前」は var"##XXX" という形で管理されており、XXX には任意の番号が振られている
struct var"##XXX"
    HP::Int
end 

=#

なお、Expr オブジェクト内で変数を参照したい場合は `$変数名` と記述する（**引用**）

これは、文字列に変数を埋め込む場合と同じ感覚で考えておいて概ね問題ない

#### 複数の式を構築する
上記で「適当な名前で構造体を定義する」ことは実現できた

しかし、今回行いたいことは「適当な名前で構造体を定義して、変数に束縛する」という操作である

すなわち、2つの式を1つの式にまとめる必要がある

これを実現する前に、`Expr` の構造を知っておく必要がある

`Expr` は `head::Symbol` と `args::Array{Any}` の2つの部分で構成されており、`head` 部分はその式の種類を表している

例えば、`a = 1` という式の場合の `head` は `=` というシンボルになり、`function f(x) x + 1 end` という式の場合の `head` は `function` というシンボルになる

複数の式をまとめる役割をもつ式としては、`block` や `toplevel` などがあり、これらの種類の Expr オブジェクトを使うことで、複数の式をまとめることができる

今回の場合、「変数に束縛する」際の変数はトップレベル変数（実行したスコープ内で参照可能な変数）であって欲しいため、`toplevel` 式を使うことになる

In [14]:
"""
以下のようなコードを生成・実行するマクロ

```
struct var"##XXX"
    HP::Int
end

キャラクター = var"##XXX"
```
"""
macro structure()
    一時的な構造体の名前 = gensym()

    Expr(:toplevel,
        :(
            struct $一時的な構造体の名前
                HP::Int
            end
        ),
        :(
            キャラクター = $一時的な構造体の名前
        )
    ) |> eval # トップレベルコードとして実行したい場合は、その場で即座に eval を実行する必要がある
end

# コード生成・実行
@structure

# => キャラクター 変数に struct var"##XXX" が束縛されているはず
キャラクター

var"##295"

In [15]:
キャラクター(100) |> dump

var"##295"
  HP: Int64 100


#### マクロ引数
今のままでは、変数名や構造体の構造等が固定されていて、いまいち使えない

そのため、マクロに引数に変数名や構造体の構造等を渡せるように変更する

今回は、以下のような形でマクロを実行できるように設計する

```julia
"""
マクロ実行時の引数は通常の関数と同じように @マクロ(引数1, 引数2, ...) という形でも書けるが、スペース区切りで書くこともできる
以下の書き方の場合は

@structure(構造体を束縛する変数名, begin
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end)

と書いているのと同等
"""

@structure 構造体を束縛する変数名 begin
    # 構造体の構造
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

"""
=> 上記マクロの実行結果が、以下のコードの実行結果と同一になるようにする

struct var"##XXX"
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

構造体を束縛する変数名 = var"##XXX"
"""
```

In [8]:
macro structure(構造体を束縛する変数名, 構造体定義)
    一時的な構造体の名前 = gensym()

    Expr(:toplevel,
        # 構造体定義の式は Expr(:struct, is_mutable::Bool, 構造体名::Symbol, 内部構造::Expr) で定義される
        # 構造体内部構造の式は Expr(:block, 内部構造式::Array{Expr}) で定義される
        # 引数.構造体定義は begin ... end 式を想定しているため Expr(:block, Array{Expr}(...))
        ## => 構造体内部構造の式として 引数.構造体定義 をそのまま渡せば構造体定義できる
        Expr(:struct, false, 一時的な構造体の名前, 構造体定義),

        # 構造体を束縛する変数名 に 一時的な構造体 として定義した構造体を束縛する式
        :(
            $構造体を束縛する変数名 = $一時的な構造体の名前
        )
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

# キャラクター 変数に構造体定義を束縛
@structure キャラクター begin
    名前::String
    HP::Int
    攻撃力::Int
    防御力::Int
end

#=> キャラクター 変数に
    struct var"##XXX"
        名前::String
        HP::Int
        攻撃力::Int
        防御力::Int
    end
が束縛されているはず
=#
dump(キャラクター)

var"##298" <: Any
  名前::String
  HP::Int64
  攻撃力::Int64
  防御力::Int64


In [9]:
キャラクター("勇者", 30, 20, 10) |> dump

var"##298"
  名前: String "勇者"
  HP: Int64 30
  攻撃力: Int64 20
  防御力: Int64 10


## デフォルトパラメータ付き構造体

ここまでで一通り、再定義可能な構造体を宣言するマクロは出来上がった

ここからは、さらに一歩進んでマクロの完成度を上げていく

Julia の構造体は、基本的に宣言時点ではメンバ変数に値を設定することはできない

しかし、`Base.@kwdef` マクロを利用すると、以下のようにメンバ変数のデフォルト値を設定することができる

```julia
Base.@kwdef struct キャラクター
    名前::String = "勇者"
    HP::Int = 30
    攻撃力::Int = 10
    防御力::Int = 10
end

キャラクター()

#=> struct キャラクター
  名前: String "勇者"
  HP: Int64 30
  攻撃力: Int64 10
  防御力: Int64 10
=#
```

ここまで作ってきた `@structure` マクロを、このデフォルトパラメータ付き構造体の宣言に対応させてみる

まずは、デフォルトパラメータ付き構造体の宣言式の Expr オブジェクトの構造を確認する

In [11]:
:(
    Base.@kwdef struct キャラクター
        名前::String = "勇者"
    end
) |> dump

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Expr
      head: Symbol .
      args: Array{Any}((2,))
        1: Symbol Base
        2: QuoteNode
          value: Symbol @kwdef
    2: LineNumberNode
      line: Int64 2
      file: Symbol In[11]
    3: Expr
      head: Symbol struct
      args: Array{Any}((3,))
        1: Bool false
        2: Symbol キャラクター
        3: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 3
              file: Symbol In[11]
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Expr
                2: String "勇者"


上記の通り、なかなか複雑な Expr オブジェクトになっている

こういった場合は、Expr オブジェクトを直接記述することに固執せず、プログラムコードの文字列から `Meta.parse` 関数を通して Expr オブジェクトを作ることを考えたほうがシンプルに書ける

ただし、`Meta.parse` 関数が分解できるのは1つの式のみであるため、複数の式を記述したい場合は `;` で区切って一行で書く必要がある

また、`gensym()` で生成した任意シンボルは、文字列化すると `var"##XXX"` ではなく、シンボルID `##XXX` の形に変換されることに注意する

In [1]:
#=
```
Base.@kwdef struct var"##XXX"
    HP::Int = 100
end

キャラクター = var"##XXX"
```

のようなコードの Expr オブジェクトを生成したい場合
=#
sym = gensym()
Meta.parse("Base.@kwdef struct var\"$sym\" HP::Int = 100 end; キャラクター = var\"$sym\"")

:($(Expr(:toplevel, :(#= none:1 =# Base.@kwdef struct var"##291"
          #= none:1 =#
          HP::Int = 100
      end), :(キャラクター = var"##291"))))

In [2]:
"""
再定義可能なデフォルトパラメータ付き構造体を定義するマクロ

```
@structure キャラクター begin
    名前::String = "勇者"
    HP::Int = 100
end

#=> 以下のコードに展開されて実行される
Base.@kwdef struct var"##XXX"
    名前::String = "勇者"
    HP::Int = 100
end
キャラクター = var"##XXX"
=#
```
"""
macro structure(構造体を束縛する変数名, 構造体定義)
    一時的な構造体の名前 = "var\"$(gensym())\""

    # 引数.構造体定義 は begin ... end ブロック式が渡されることを想定
    ## => ブロック内部の式を `;` 区切りのプログラムコード文字列に変換
    構造体内部コード = join(構造体定義.args, ";")

    # プログラムコード文字列から Expr 生成
    Meta.parse(
        "Base.@kwdef struct $一時的な構造体の名前 $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

# キャラクター 変数に構造体定義を束縛
@structure キャラクター begin
    名前::String = "勇者"
    HP::Int = 30
    攻撃力::Int = 20
    防御力::Int = 10
end

キャラクター() |> dump

var"##292"
  名前: String "勇者"
  HP: Int64 30
  攻撃力: Int64 20
  防御力: Int64 10


### マクロ引数を構造体宣言式に変更
最後に、可変構造体 (mutable struct) も普通の構造体 (struct) も同じように書けるようにマクロ定義を思い切って変えてしまう

すなわち、マクロ呼び出しを以下のように書けるようにしたい

```julia
# 再定義可能な struct
@restruct struct キャラクター
    HP::Int = 100
end

#=> 以下のように展開される
Base.@kwdef struct var"##XXX"
    HP::Int = 100
end
キャラクター = var"##XXX"
=#


# 再定義可能な mutable struct
@restruct mutable struct キャラクター
    HP::Int = 100
end

#=> 以下のように展開される
Base.@kwdef mutable struct var"##XXX"
    HP::Int = 100
end
キャラクター = var"##XXX"
=#
```

これを実現するためには、マクロの引数を構造体宣言式にした上で、その式を分解 => 再構築する必要がある

改めて、構造体宣言式の Expr オブジェクトの中身を確認しておく

In [6]:
:(struct キャラクター HP::Int = 100 end) |> dump

Expr
  head: Symbol struct
  args: Array{Any}((3,))
    1: Bool false
    2: Symbol キャラクター
    3: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 1
          file: Symbol In[6]
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol HP
                2: Symbol Int
            2: Int64 100


In [7]:
:(mutable struct キャラクター HP::Int = 100 end) |> dump

Expr
  head: Symbol struct
  args: Array{Any}((3,))
    1: Bool true
    2: Symbol キャラクター
    3: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 1
          file: Symbol In[7]
        2: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Expr
              head: Symbol ::
              args: Array{Any}((2,))
                1: Symbol HP
                2: Symbol Int
            2: Int64 100


すなわち、マクロ引数に渡されてくる構造体宣言式は

```julia
Expr(:struct, is_mutable::Bool, 構造体名::Symbol, 構造体内部構造::Expr)
```

である

これをうまく分解して、求めるプログラムコードに変換すれば良い

In [10]:
"""
再定義可能なデフォルトパラメータ付き構造体を定義するマクロ

```
@restruct mutable struct キャラクター
    名前::String = "勇者"
    HP::Int = 100
end

#=> 以下のコードに展開されて実行される
Base.@kwdef mutable struct var"##XXX"
    名前::String = "勇者"
    HP::Int = 100
end
キャラクター = var"##XXX"
=#
```
"""
macro restruct(構造体宣言式)
    一時的な構造体の名前 = "var\"$(gensym())\""

    #= 引数.構造体宣言式.args
        [1]: is_mutable::Bool
        [2]: 構造体名::Symbol
        [3]: 構造体内部構造::Expr
            構造体内部構造.args
                [...]: 内部構造式の配列
    =#
    mutable = 構造体宣言式.args[1] ? "mutable" : ""
    構造体を束縛する変数名 = 構造体宣言式.args[2]
    構造体内部コード = join(構造体宣言式.args[3].args, ";")

    Meta.parse(
        "Base.@kwdef $mutable struct $一時的な構造体の名前 $構造体内部コード end; $構造体を束縛する変数名 = $一時的な構造体の名前"
    ) |> eval # トップレベルコードとしてその場で即座に eval を実行
end

# キャラクター 変数に可変構造体定義を束縛
@restruct mutable struct キャラクター
    名前::String = "勇者"
    HP::Int = 30
    攻撃力::Int = 20
    防御力::Int = 10
end

勇者 = キャラクター()
dump(勇者)

var"##295"
  名前: String "勇者"
  HP: Int64 30
  攻撃力: Int64 20
  防御力: Int64 10


In [11]:
# 可変構造体になっているはずなため、メンバ変数を再代入可能か確認
勇者.名前 = "プレイヤー"
勇者.HP = 100
勇者.攻撃力 = 25
勇者.防御力 = 15

dump(勇者)

var"##295"
  名前: String "プレイヤー"
  HP: Int64 100
  攻撃力: Int64 25
  防御力: Int64 15


これで、再定義可能な構造体の宣言が楽に行えるようになった